In [3]:
from typing import List
import sys
sys.path.append('C:/Users/TomFi/Desktop/Cours/Projet IA/neuroforest-master')
import numpy as np
from IPython import display
from PIL import Image, ImageDraw, ImageFont

from main import dataloader, to_vect, Coordinates
import matplotlib.pyplot as plt
import torch

In [9]:
def create_batches(dataloader, session_type: str, batch_size: int):
    trajectories = []

    for sample in dataloader:
        session = sample[session_type]
        if session is not None:
            player_coords = np.array([c["coord"].to_vect() for c in session.player_coords])
            player_coords = player_coords[:, [0, 2]]  # Keep only x and z coordinates
            trajectories.append(player_coords)

    # Determine the minimum length of the trajectories
    min_length = min(len(traj) for traj in trajectories)

    # Truncate the trajectories to the minimum length
    truncated_trajectories = np.array([traj[:min_length] for traj in trajectories])
    tensor_trajectories = torch.tensor(truncated_trajectories, dtype=torch.float32)

    # Create batches
    num_batches = len(tensor_trajectories) // batch_size
    batches = tensor_trajectories[:num_batches * batch_size].view(num_batches, batch_size, min_length, 2)

    return batches

# Create batches for "uniform" and "patchy" session types
batch_size = 1
uniform_batches = create_batches(dataloader, "uniform", batch_size)
patchy_batches = create_batches(dataloader, "patchy", batch_size)

print(f"Uniform batches shape: {uniform_batches.shape}")
print(f"Patchy batches shape: {patchy_batches.shape}")

Uniform batches shape: torch.Size([39, 1, 29999, 2])
Patchy batches shape: torch.Size([38, 1, 29998, 2])
